### Import

In [ ]:
# Import the NumPy library for matrix manipulation
import numpy as np

# Simplified DES

In [ ]:
def permute(key, perm):
    """Applies a permutation to a key."""
    permuted_key = ""
    for i in perm:
        permuted_key += key[i - 1]
    return permuted_key

def left_shift(key, shifts):
    """Performs a circular left shift on a key."""
    return key[shifts:] + key[:shifts]

def generate_subkeys(key):
    """Generates K1 and K2 subkeys from the 10-bit key."""
    key = permute(key, P10)
    key = left_shift(key, 1)
    k1 = permute(key, P8)
    key = left_shift(key, 2)
    k2 = permute(key, P8)
    return k1, k2

def f_k(data, subkey):
    """The f_k function in S-DES."""
    data = permute(data, E_P)
    data = xor(data, subkey)
    row0 = int(data[0] + data[3], 2)
    col0 = int(data[1] + data[2], 2)
    row1 = int(data[4] + data[7], 2)
    col1 = int(data[5] + data[6], 2)
    sbox_out = bin(S0[row0][col0])[2:].zfill(2) + bin(S1[row1][col1])[2:].zfill(2)
    return permute(sbox_out, P4)

def xor(a, b):
    """Performs bitwise XOR on two strings."""
    result = ""
    for i in range(len(a)):
        if a[i] == b[i]:
            result += "0"
        else:
            result += "1"
    return result

def sdes_encrypt(plaintext, key):
    """Encrypts plaintext using S-DES and the given key."""
    k1, k2 = generate_subkeys(key)
    data = permute(plaintext, IP)
    data = xor(data[:4], f_k(data[4:], k1)) + data[4:]
    data = data[4:] + data[:4]  # Swap
    data = xor(data[:4], f_k(data[4:], k2)) + data[4:]
    ciphertext = permute(data, IP_inv)
    return ciphertext

def sdes_decrypt(ciphertext, key):
    """Decrypts ciphertext using S-DES and the given key."""
    k1, k2 = generate_subkeys(key)
    data = permute(ciphertext, IP)
    data = xor(data[:4], f_k(data[4:], k2)) + data[4:]
    data = data[4:] + data[:4]  # Swap
    data = xor(data[:4], f_k(data[4:], k1)) + data[4:]
    plaintext = permute(data, IP_inv)
    return plaintext

## Example of S-DES

In [ ]:
# Example usage of sdes_encrypt and sdes_decrypt
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
IP = [2, 6, 3, 1, 4, 8, 5, 7]
IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]
E_P = [4, 1, 2, 3, 2, 3, 4, 1]
P4 = [2, 4, 3, 1]
S0 = [[1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]]
S1 = [[0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]]


plaintext = "10111101"
key = "1010000010"
ciphertext = sdes_encrypt(plaintext, key)
decrypted_text = sdes_decrypt(ciphertext, key)

print("Plaintext:", plaintext)
print("Key:", key)
print("Ciphertext:", ciphertext)
print("Decrypted text:", decrypted_text)

# Enhanced S-DES
This section contains the necessary functions for the transformations and shifting prior to applying the S-DES algorithm.


In [12]:
# Define the transposition key and calculate the number of columns
transp_key = "emirube"
n_columns = len(transp_key)
n_rows = 0 # Initially, the number of rows is unknown

# Random list that defines the column order
random_list = [3, 4, 5, 6, 2, 0, 1]
print(random_list)

# Function to reorder the columns of a matrix based on a given order
def MR(arr, order):
    # Initialize an empty string to hold the resulting text
    result_string = ""
    
    # Iterate through the columns based on the provided order
    for col in order:
        # Extract the entire column and append the values row by row
        for row in arr[:, col]:
            result_string += row  # Concatenate each element
    
    return result_string 

# Function to revert the column reordering of a matrix
def revert_MR(arr, order):
    # Step 1: Convert the array to a string by concatenating all values row by row
    text = ''.join(''.join(row) for row in arr)
    
    # Initialize a result array (same shape as the original)
    n_rows, n_columns = arr.shape
    result_matrix = np.full((n_rows, n_columns), '', dtype='<U1')  # Create empty matrix to store result
    
    # Step 2: Fill the result matrix column by column, based on the 'order'
    idx = 0  # Index to keep track of characters in the string
    for col in order:
        for row in range(n_rows):
            if arr[row, col] != '':  # Only fill in non-empty positions
                if idx < len(text):  # Make sure we don't go out of bounds
                    result_matrix[row, col] = text[idx]
                    idx += 1
    
    # Step 3: Convert the result matrix back into a single string
    result_string = ''.join(''.join(row) for row in result_matrix)
    
    return result_string

# Function to create a matrix based on the provided text
def matrixmaker(text):
    # Initialize the matrix with empty strings
    matrix = np.full((n_rows, n_columns), '', dtype='<U1')

    # Fill the matrix row by row
    for i in range(n_rows):
        # Slice the text list to get the next row
        start_idx = i * n_columns
        end_idx = start_idx + n_columns
        row = text[start_idx:end_idx]  # Get the characters for the current row
        
        # Assign the sliced row to the matrix
        matrix[i, :len(row)] = list(row)  # Only fill as many columns as the row has characters
    
    return matrix

def shift_rows_left(matrix):
    n_rows, n_columns = matrix.shape
    
    # Create a new matrix to store the result
    shifted_matrix = np.empty_like(matrix)
    
    # Perform the row shifting to the left (ignoring empty strings '')
    for i in range(n_rows):
        # Get the non-empty values in the row
        non_empty_values = [val for val in matrix[i] if val != '']
        
        # Perform the circular shift on the non-empty values
        shift = i % len(non_empty_values) if len(non_empty_values) > 0 else 0
        
        # Roll (shift) the non-empty values to the left
        shifted_non_empty_values = np.roll(non_empty_values, -shift)
        
        # Fill the shifted matrix, preserving positions of empty ('') values
        idx = 0  # Index for non-empty values
        for j in range(n_columns):
            if matrix[i, j] != '':
                shifted_matrix[i, j] = shifted_non_empty_values[idx]
                idx += 1
            else:
                shifted_matrix[i, j] = ''  # Leave the empty string as is
    
    return shifted_matrix

def shift_rows_right(matrix):
    n_rows, n_columns = matrix.shape
    
    # Create a new matrix to store the result
    reverted_matrix = np.empty_like(matrix)
    
    # Perform the reverse row shifting (circular shift, ignoring '')
    for i in range(n_rows):
        # Get the non-empty values in the row
        non_empty_values = [val for val in matrix[i] if val != '']
        
        # Perform the circular shift on the non-empty values
        shift = i % len(non_empty_values) if len(non_empty_values) > 0 else 0
        
        # Roll (shift) the non-empty values to the right
        shifted_non_empty_values = np.roll(non_empty_values, shift)
        
        # Fill the reverted matrix, preserving positions of empty ('') values
        idx = 0  # Index for non-empty values
        for j in range(n_columns):
            if matrix[i, j] != '':
                reverted_matrix[i, j] = shifted_non_empty_values[idx]
                idx += 1
            else:
                reverted_matrix[i, j] = ''  # Leave the empty string as is
    
    return reverted_matrix

[3, 4, 5, 6, 2, 0, 1]


## Example of Enhanced S-DES

In [15]:
# plaintext = input("Enter plaintext:")
plaintext = "are you coming to the toga party"
plaintext = plaintext.replace(" ", "")

ptcode = [char for char in plaintext]

In [16]:
# Calculate the number of rows needed based on the pre-shared number of columns and text
n_rows = (len(ptcode) + n_columns - 1) // n_columns  # Ceiling division

ogmatrix = matrixmaker(ptcode)
print(ogmatrix)
print()
firstround = MR(ogmatrix, random_list)
print("First round:", firstround)

fstmatrix = matrixmaker(firstround)
print(fstmatrix)
print()
secondround = MR(fstmatrix, random_list)
print("Second round:", secondround)

trans_output = matrixmaker(secondround)
print("Transposition output:")
print(trans_output)

[['a' 'r' 'e' 'y' 'o' 'u' 'c']
 ['o' 'm' 'i' 'n' 'g' 't' 'o']
 ['t' 'h' 'e' 't' 'o' 'g' 'a']
 ['p' 'a' 'r' 't' 'y' '' '']]

First round: ynttogoyutgcoaeieraotprmha
[['y' 'n' 't' 't' 'o' 'g' 'o']
 ['y' 'u' 't' 'g' 'c' 'o' 'a']
 ['e' 'i' 'e' 'r' 'a' 'o' 't']
 ['p' 'r' 'm' 'h' 'a' '' '']]

Second round: tgrhocaagoooatttemyyepnuir
Transposition output:
[['t' 'g' 'r' 'h' 'o' 'c' 'a']
 ['a' 'g' 'o' 'o' 'o' 'a' 't']
 ['t' 't' 'e' 'm' 'y' 'y' 'e']
 ['p' 'n' 'u' 'i' 'r' '' '']]


In [17]:
shifted_output = shift_rows_left(trans_output)
print("Shifted output:")
print(shifted_output)

Shifted output:
[['t' 'g' 'r' 'h' 'o' 'c' 'a']
 ['g' 'o' 'o' 'o' 'a' 't' 'a']
 ['e' 'm' 'y' 'y' 'e' 't' 't']
 ['i' 'r' 'p' 'n' 'u' '' '']]


In [18]:
reverted_output = shift_rows_right(shifted_output)
print("Reverted output:")
print(reverted_output)

Reverted output:
[['t' 'g' 'r' 'h' 'o' 'c' 'a']
 ['a' 'g' 'o' 'o' 'o' 'a' 't']
 ['t' 't' 'e' 'm' 'y' 'y' 'e']
 ['p' 'n' 'u' 'i' 'r' '' '']]


In [19]:
rev_round = revert_MR(reverted_output, random_list)
print(rev_round)
rev1 = matrixmaker(rev_round)
print(rev1)
print()

rev_round2 = revert_MR(rev1, random_list)
print(rev_round2)
rev2 = matrixmaker(rev_round2)
print(rev2)

ynttogoyutgcoaeieraotprmha
[['y' 'n' 't' 't' 'o' 'g' 'o']
 ['y' 'u' 't' 'g' 'c' 'o' 'a']
 ['e' 'i' 'e' 'r' 'a' 'o' 't']
 ['p' 'r' 'm' 'h' 'a' '' '']]

areyoucomingtothetogaparty
[['a' 'r' 'e' 'y' 'o' 'u' 'c']
 ['o' 'm' 'i' 'n' 'g' 't' 'o']
 ['t' 'h' 'e' 't' 'o' 'g' 'a']
 ['p' 'a' 'r' 't' 'y' '' '']]


Aquí empieza el cambio a binario para empezar el S-DES

In [20]:
# Flatten the shifted matrix into a long list
flattened_list = shifted_output.flatten().tolist()

# Remove all '' values
filtered_list = [val for val in flattened_list if val != '']

# Print the flattened list
print(filtered_list)

['t', 'g', 'r', 'h', 'o', 'c', 'a', 'g', 'o', 'o', 'o', 'a', 't', 'a', 'e', 'm', 'y', 'y', 'e', 't', 't', 'i', 'r', 'p', 'n', 'u']


In [21]:
binary_list = [format(ord(char), '08b') for char in filtered_list]
print(binary_list)

['01110100', '01100111', '01110010', '01101000', '01101111', '01100011', '01100001', '01100111', '01101111', '01101111', '01101111', '01100001', '01110100', '01100001', '01100101', '01101101', '01111001', '01111001', '01100101', '01110100', '01110100', '01101001', '01110010', '01110000', '01101110', '01110101']
